# AWS Glue Studio Notebook



Conexión con la base de datos en Glue Catalog y carga de datos

In [15]:
import sys
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.transforms import *

glueContext = GlueContext(SparkContext.getOrCreate())

# Cargar datos de DynamoDB
dyf = glueContext.create_dynamic_frame.from_catalog(database="glue_joan", table_name="Peliculas_S3D2_xideral")

df = dyf.toDF()



Hacer consulta de la clasificacion mas repetida de la tabla y cargar los datos en un json alojado en un s3 bucket

In [16]:
result = df.groupBy("clasificacion").count().orderBy("count", ascending=False).limit(1)

result.write.mode("overwrite").json("s3://joanmoviesbucket/clasificaciones/")

Funcion para detectar el tipo de genero de cada pelicula con respecto a su titulo y carga del nuevo campo de genero a una nueva base de datos en DynamoDb

In [22]:
import sys
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *

# Crear el contexto de Glue
glueContext = GlueContext(SparkContext.getOrCreate())

# Cargar datos desde DynamoDB usando el Glue Catalog
dyf = glueContext.create_dynamic_frame.from_catalog(
    database="glue_joan",  # Nombre de la base de datos del Glue Catalog
    table_name="Peliculas_S3D2_xideral"  # Nombre de la tabla catalogada
)

# Convertir el DynamicFrame a un DataFrame de Spark para aplicar operaciones de agregación
df = dyf.toDF()

# Lista de palabras clave por género
generos = {
    "Terror": ["horror", "miedo", "scream", "pesadilla", "exorcista", "terror"],
    "Comedia": ["comedia", "divertido", "loco", "gracioso", "risa", "diversión"],
    "Acción": ["acción", "explosión", "batalla", "venganza", "lucha", "superhéroe"],
    "Drama": ["drama", "historia", "tragedia", "emocional", "triste", "familia"],
    "Ciencia Ficción": ["alien", "futuro", "robot", "galaxia", "espacio", "científico"],
    "Animación": ["animado", "dibujos", "pixar", "disney", "animación", "familia"]
}

# Función para detectar el género basado en el título de la película
def detectar_genero(titulo):
    for genero, palabras in generos.items():
        if any(palabra.lower() in titulo.lower() for palabra in palabras):
            return genero
    return "Desconocido"

detectar_genero_udf = udf(detectar_genero, StringType())

# Aplicar el UDF a la columna 'nombre' para obtener la columna 'genero'
df = df.withColumn("genero", detectar_genero_udf(df["nombre"]))

# Convertir de nuevo a DynamicFrame para escribir en DynamoDB
dyf_generos = DynamicFrame.fromDF(df, glueContext, "dyf_generos")

glueContext.write_dynamic_frame.from_options(
    dyf_generos, 
    connection_type="dynamodb", 
    connection_options={"tableName": "Peliculas_Generos_Joan", "createTable": "false"}
)

print("Generos agregados y guardados en la nueva tabla de DynamoDB")




Generos agregados y guardados en la nueva tabla de DynamoDB
